In [2]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests

import dotenv

import IPython
from IPython.display import HTML, Markdown, display

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.3
BeautifulSoup   4.13.3


In [4]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [5]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=50,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom) for everything goes to 1.', display_name='SkynetAndChill.com', followers_count=328, follows_count=347, indexed_at='2024-11-21T15:40:08.470Z', joined_via_starter_p

In [6]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [7]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [8]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [9]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [10]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [11]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [12]:
# import sqlite3
# import tldextract

# # Path to your SQLite database
# db_path = "articles.db"

# # Connect to the database
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# # Fetch all rows (id, hostname)
# cur.execute("SELECT id, hostname FROM sites")
# rows = cur.fetchall()

# for row in rows:
#     site_id, hostname = row
#     # Extract the registered domain (e.g., 'bbc.co.uk', '9to5mac.com')
#     extracted = tldextract.extract(hostname)
#     if extracted.domain and extracted.suffix:
#         registered_domain = f"{extracted.domain}.{extracted.suffix}"
#         # Update the domain column
#         print(f"{hostname} -> {registered_domain}")
#         cur.execute(
#             "UPDATE sites SET domain = ? WHERE id = ?",
#             (registered_domain, site_id))
#     else:
#         registered_domain = None  # fallback if parsing fails

# # Commit changes and close
# conn.commit()
# conn.close()

In [13]:
imgdir = 'tmp'  # for images
delete_files(imgdir)

conn = sqlite3.connect('articles.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname: row.site_name for row in sites_df.itertuples()}
conn.close()


In [14]:
def parse_feed():
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):

        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.rstrip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                parsed_url = urlparse(post_url)
                hostname = parsed_url.hostname
                site_name = sites_dict.get(hostname, hostname)

            md_str = f"[{post_str}]({post_url})  - {site_name}"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed()
len(display_dict)

# md_output += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"
# final_display_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"
# display(Markdown(final_display_str))


50

In [15]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))


0. 
ngl, it generates an excellent fully automated AI news summary tho



---

1. [Microsoft launched a scientific AI that screened 100s of thousands of candidates and discovered a potential new coolant in 200 hours instead of years](https://venturebeat.com/ai/microsoft-just-launched-an-ai-that-discovered-a-new-chemical-in-200-hours-instead-of-years/)  - VentureBeat![image](tmp/Image1.jpg)



---

2. [Comparing ChatGPT's grammatical judgment with linguists and lay people finds it's pretty good. Not that clear which GPT model. ChatGPT is not great at assigning calibrated scores, comparing 2 sentence is better. (I believe they found that in the study)](https://www.nature.com/articles/s41599-025-04907-8)  - Nature![image](tmp/Image2.jpg)



---

3. [techcrunch.com/2025/05/19/k...](https://techcrunch.com/2025/05/19/klarnas-revenue-per-employee-soars-to-nearly-1m-thanks-to-ai-efficiency-push/)  - TechCrunch![image](tmp/Image3.jpg)



---

4. [Klarna CEO says AI push may have gone too far, wants more humans now. Pushed revenue per worker way up, but credit quality is down. Who could have predicted someone who buys a DoorDash burrito on a payment plan could be a bad credit risk?](https://futurism.com/klarna-openai-humans-ai-back)  - Futurism![image](tmp/Image4.jpg)



---

5. [Stratechery's Ben Thompson Interviews Nvidia CEO Jensen Huang About Chip Controls, AI Factories, and Enterprise Pragmatism](https://stratechery.com/2025/an-interview-with-nvidia-ceo-jensen-huang-about-chip-controls-ai-factories-and-enterprise-pragmatism/)  - Stratechery by Ben Thompson![image](tmp/Image5.jpg)



---

6. [Google releases its NotebookLM mobile app](https://www.theverge.com/news/669828/google-releases-its-notebooklm-mobile-app)  - The Verge![image](tmp/Image6.jpg)



---

7. [Hyper-contextual, (personalized with facial recognition?) GenAI creative is just about to change the face of out-of-home advertising](https://www.thedrum.com/opinion/2025/05/19/genai-creative-just-about-change-the-face-out-home-advertising)  - The Drum![image](tmp/Image7.jpg)



---

8. [Microsoft's big event was all about the 'explosion' of AI agents](https://www.businessinsider.com/microsoft-build-keynote-2025-ai-agent-use-doubled-2025-5)  - Business Insider![image](tmp/Image8.jpg)



---

9. [At Build 2025, Microsoft announced major updates to its Copilot Studio platform, including multi-agent computer use that enables different AI agents to collaborate on complex business tasks, integration with WhatsApp, Python-powered analytics, and new developer/admin tools.](https://venturebeat.com/ai/microsoft-just-taught-its-ai-agents-to-talk-to-each-other-and-it-could-transform-how-we-work/)  - VentureBeat![image](tmp/Image9.jpg)



---

10. [Baiont’s Feng Ji: Quant managers who don’t adopt AI will be eliminated by the market](https://www.ft.com/content/8eada27c-fa22-4406-9e2b-5034b92642fb)  - Financial Times![image](tmp/Image10.jpg)



---

11. [Fair analogy, you have an autopilot and also 2 highly trained pilots monitoring it and turning it on and off at appropriate times. But also, the amount of regulatory oversight over aircraft manufacturing and operation is extremely intense, and every rule was written in blood.](https://www.forbes.com/councils/forbestechcouncil/2025/05/19/if-you-can-trust-an-airplanes-autopilot-you-can-trust-ai/)  - Forbes![image](tmp/Image11.jpg)



---

12. 




---

13. 
tfw I realize I just sent 10,000 prompts to 4.1 instead of 4.1-mini and it's gonna cost me \\$20 instead of \\$4 and take a little while. oops!



---

14. [Grok’s ‘white genocide’ meltdown nods to the real dangers of the AI arms race 

There is no unbiased 'view from nowhere'; AI inevitably confirms the biases of its creators, trainers, labelers; in the hands of intemperate, greedy people, it will heavy-handedly favor their agenda and interests.](https://www.cnn.com/2025/05/20/business/grok-genocide-ai-nightcap)  - CNN![image](tmp/Image14.jpg)



---

15. [Driverless vehicles that respond more like humans will cause less harm during road crashes, according to research](https://www.ft.com/content/4eac53c6-85f6-4fe2-8d9c-2e4503473e69)  - Financial Times![image](tmp/Image15.jpg)



---

16. ['Godfather of AI' Geoffrey Hinton says he trusts his chatbot more than he should](https://www.businessinsider.com/geoffrey-hinton-godfather-ai-gpt4-openai-2025)  - Business Insider![image](tmp/Image16.jpg)



---

17. ["the moment I realized AI could code better than me"](https://www.reddit.com/r/ChatGPT/comments/1kq8t4t/the_moment_i_realized_ai_could_code_better_than_me/)  - Reddit

---

18. [Windows is getting support for the MCP, the ‘USB-C of AI apps’](https://www.theverge.com/news/669298/microsoft-windows-ai-foundry-mcp-support)  - The Verge![image](tmp/Image18.jpg)



---

19. 
like all that AI news to catch up on



---

20. [LLMs can beat humans in debates - if they get personal info.

If your candidate's chatbot knows your personal info, maybe it can persuade you better.](https://www.theregister.com/2025/05/19/when_llms_get_personal_info/)  - The Register

---

21. [Time saved by AI offset by new work created, study suggests](https://arstechnica.com/ai/2025/05/time-saved-by-ai-offset-by-new-work-created-study-suggests/)  - Ars Technica![image](tmp/Image21.jpg)



---

22. [SAG-AFTRA filed an unfair labor practice complaint over Darth Vader's voice in Fortnite](https://arstechnica.com/ai/2025/05/fortnites-ai-darth-vader-spawns-unfair-labor-practice-charge-from-voice-union/)  - Ars Technica![image](tmp/Image22.jpg)



---

23. [Trump signs the Take It Down Act criminalizing AI deepfakes today. Hopefully it won't have unintended consequences like putting people in jail for Trump caricatures, sites not allowing image/video posting.](https://www.engadget.com/big-tech/trump-will-sign-the-take-it-down-act-criminalizing-ai-deepfakes-today-184358916.html)  - Engadget![image](tmp/Image23.jpg)



---

24. [Microsoft did some stuff to support registering your agents, monitoring them, and red-teaming them](https://www.zdnet.com/article/microsoft-makes-building-trustworthy-ai-agents-easier-and-more-secure/)  - ZDNET![image](tmp/Image24.jpg)



---

25. 
today I was comparing Substack  v. Ghost v. Beehiiv
tomorrow I might compare NYC mayoral candidates on my hot-button issues

instead of talking to o3, I could have an agent in my browser, that queries the chatbots on the product/campaign pages and synthesizes and gives me the result

brave new world



---

26. [How Google decided to give websites no opt-out/in for AI search](https://www.bloomberg.com/news/articles/2025-05-19/google-gave-sites-little-choice-in-using-data-for-ai-search?srnd=phx-ai)  - Bloomberg

---

27. [Best free AI courses - www.fastcompany.com/91332955/cou...
See also
mashable.com/article/free...
tech.yahoo.com/ai/articles/...
or anything from deeplearning.ai
www.deeplearning.ai/courses/](https://www.fastcompany.com/91332955/coursera-best-free-ai-courses)  - Fast Company

---

28. [A review of brain-computer interfaces, with number of studies and patients set to rise](https://www.wsj.com/tech/brain-implant-musk-als-tbi-neuralink-f733998f)  - The Wall Street Journal

---

29. [New Orleans called out for sketchiest use of facial recognition yet in the US](https://arstechnica.com/tech-policy/2025/05/cops-pause-use-of-flawed-ai-cameras-secretly-monitoring-streets-for-suspects/)  - Ars Technica![image](tmp/Image29.jpg)



---

30. [NLWeb lets you add a chatbot to your web page, and agents would know how to talk to it.](https://techcrunch.com/2025/05/19/nlweb-is-microsofts-project-to-bring-more-chatbots-to-webpages/)  - TechCrunch![image](tmp/Image30.jpg)



---

31. [GitHub allows you to assign issues to a coding agent that will go away and think and submit a PR](https://www.theverge.com/news/669339/github-ai-coding-agent-fix-bugs)  - The Verge![image](tmp/Image31.jpg)



---

32. [An interview with Microsoft CTO Kevin Scott on the birth of the agentic web](https://www.theverge.com/decoder-podcast-with-nilay-patel/669409/microsoft-cto-kevin-scott-interview-ai-natural-language-search-openai)  - The Verge![image](tmp/Image32.jpg)



---

33. [China deploys world’s biggest fleet of driverless mining trucks. Robot miners seems like a pretty reasonable use case for safety, health, productivity.](https://www.scmp.com/tech/big-tech/article/3310470/china-deploys-worlds-largest-fleet-driverless-mining-trucks-powered-huawei-tech)  - South China Morning Post![image](tmp/Image33.jpg)



---

34. [Sundar Pichai acknowledged competition between the top companies developing AI, but he said they all are "going to do well."](https://www.businessinsider.com/sundar-pichai-doesnt-see-ai-race-as-win-lose-situation-2025-5)  - Business Insider![image](tmp/Image34.jpg)



---

35. [After Karen Hao's first story on OpenAI in 2020, they wouldn't talk to her for 3 years.](https://www.technologyreview.com/2025/05/19/1116614/hao-empire-ai-openai/)  - MIT Technology Review![image](tmp/Image35.jpg)



---

36. [On 60 Minutes, Anduril gave a behind-the-scenes look at its autonomous AI fighter built as a wingman for America's most fearsome aircraft](https://www.businessinsider.com/anduril-fury-drone-behind-scenes-us-air-force-collaborative-aircraft-2025-5)  - Business Insider![image](tmp/Image36.jpg)



---

37. [AI is coming for call centers](https://www.latimes.com/business/story/2025-05-19/call-centers-replaced-many-doctors-receptionists-now-ai-is-coming-for-call-centers)  - Los Angeles Times

---

38. [Can I copyright my Midjourney face swaps and other IP questions](https://www.cnet.com/tech/services-and-software/were-all-copyright-owners-why-you-need-to-care-about-ai-and-copyright/)  - CNET![image](tmp/Image38.jpg)



---

39. [Google's Jeff Dean predicts AI could perform at the level of a junior coder in a year](https://www.businessinsider.com/google-boss-ai-junior-coder-within-a-year-2025-5)  - Business Insider![image](tmp/Image39.jpg)



---

40. [The AI revolution changing how we predict the weather](https://www.ft.com/content/a1b6ab08-9703-45fb-b967-a40b4855bc17)  - Financial Times![image](tmp/Image40.jpg)



---

41. [John Oliver on Trump's strong-arming and legal assault on the free press](https://www.youtube.com/watch?v=07JQr5W3970)  - YouTube![image](tmp/Image41.jpg)



---

42. [Nvidia opens NVLink to 3rd-party hardware manufacturers.](https://www.theregister.com/2025/05/19/nvidia_nvlink_fusion/)  - The Register

---

43. [Nvidia launches GPU marketplace with one stop shopping for multiple clouds like CoreWeave.](https://www.wsj.com/articles/nvidia-pushes-further-into-cloud-with-gpu-marketplace-4fba6bdd)  - The Wall Street Journal

---

44. [UBS deploys AI analyst clones with Synthesia. Instead of recording a video message blast, they just write the script and send a synthesized version of themselves. I'm not sure taking a short note and turning it into this sort of video makes it something a client will pay more money for though?](https://www.ft.com/content/0916d635-755b-4cdc-b722-e32d94ae334d)  - Financial Times![image](tmp/Image44.jpg)



---

45. [Nvidia in advanced talks to invest in quantum computing startup PsiQuantum, The Information reports](https://www.reuters.com/business/nvidia-advanced-talks-invest-psiquantum-information-reports-2025-05-19/)  - Reuters

---

46. [Dario Amodei profile, how he transformed from an academic into the CEO of a \\$61 billion startup.](https://www.bloomberg.com/news/features/2025-05-19/anthropic-ceo-amodei-steers-61-billion-ai-powerhouse)  - Bloomberg

---

47. 
via @mbaram at the other place , I don't think it even includes tariffs and people getting tossed off Medicare. All these Nazi Hitler salutes and the trains don't even run on time, planes are falling out of the sky b/c destroying government capacity by design



---

48. [Now AI HR bots give you the runaround and then ghost you. “Tell me about a time when—when—let’s. Let’s circle back. Tell me about a time when—when—let’s.”](https://slate.com/life/2025/05/jobs-ai-job-hiring-character-interview.html)  - Slate Magazine![image](tmp/Image48.jpg)



---

49. [Sam Altman says critics of Trump's AI deals with Gulf nations are 'naive'](https://www.businessinsider.com/openai-sam-altman-trump-ai-deals-saudi-arabia-qatar-uae-2025-5)  - Business Insider![image](tmp/Image49.jpg)



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [16]:
stop_index = 33

In [17]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))

0. 
ngl, it generates an excellent fully automated AI news summary tho



---

1. [Microsoft launched a scientific AI that screened 100s of thousands of candidates and discovered a potential new coolant in 200 hours instead of years](https://venturebeat.com/ai/microsoft-just-launched-an-ai-that-discovered-a-new-chemical-in-200-hours-instead-of-years/)  - VentureBeat![image](tmp/Image1.jpg)



---

2. [Comparing ChatGPT's grammatical judgment with linguists and lay people finds it's pretty good. Not that clear which GPT model. ChatGPT is not great at assigning calibrated scores, comparing 2 sentence is better. (I believe they found that in the study)](https://www.nature.com/articles/s41599-025-04907-8)  - Nature![image](tmp/Image2.jpg)



---

3. [techcrunch.com/2025/05/19/k...](https://techcrunch.com/2025/05/19/klarnas-revenue-per-employee-soars-to-nearly-1m-thanks-to-ai-efficiency-push/)  - TechCrunch![image](tmp/Image3.jpg)



---

4. [Klarna CEO says AI push may have gone too far, wants more humans now. Pushed revenue per worker way up, but credit quality is down. Who could have predicted someone who buys a DoorDash burrito on a payment plan could be a bad credit risk?](https://futurism.com/klarna-openai-humans-ai-back)  - Futurism![image](tmp/Image4.jpg)



---

5. [Stratechery's Ben Thompson Interviews Nvidia CEO Jensen Huang About Chip Controls, AI Factories, and Enterprise Pragmatism](https://stratechery.com/2025/an-interview-with-nvidia-ceo-jensen-huang-about-chip-controls-ai-factories-and-enterprise-pragmatism/)  - Stratechery by Ben Thompson![image](tmp/Image5.jpg)



---

6. [Google releases its NotebookLM mobile app](https://www.theverge.com/news/669828/google-releases-its-notebooklm-mobile-app)  - The Verge![image](tmp/Image6.jpg)



---

7. [Hyper-contextual, (personalized with facial recognition?) GenAI creative is just about to change the face of out-of-home advertising](https://www.thedrum.com/opinion/2025/05/19/genai-creative-just-about-change-the-face-out-home-advertising)  - The Drum![image](tmp/Image7.jpg)



---

8. [Microsoft's big event was all about the 'explosion' of AI agents](https://www.businessinsider.com/microsoft-build-keynote-2025-ai-agent-use-doubled-2025-5)  - Business Insider![image](tmp/Image8.jpg)



---

9. [At Build 2025, Microsoft announced major updates to its Copilot Studio platform, including multi-agent computer use that enables different AI agents to collaborate on complex business tasks, integration with WhatsApp, Python-powered analytics, and new developer/admin tools.](https://venturebeat.com/ai/microsoft-just-taught-its-ai-agents-to-talk-to-each-other-and-it-could-transform-how-we-work/)  - VentureBeat![image](tmp/Image9.jpg)



---

10. [Baiont’s Feng Ji: Quant managers who don’t adopt AI will be eliminated by the market](https://www.ft.com/content/8eada27c-fa22-4406-9e2b-5034b92642fb)  - Financial Times![image](tmp/Image10.jpg)



---

11. [Fair analogy, you have an autopilot and also 2 highly trained pilots monitoring it and turning it on and off at appropriate times. But also, the amount of regulatory oversight over aircraft manufacturing and operation is extremely intense, and every rule was written in blood.](https://www.forbes.com/councils/forbestechcouncil/2025/05/19/if-you-can-trust-an-airplanes-autopilot-you-can-trust-ai/)  - Forbes![image](tmp/Image11.jpg)



---

12. 




---

13. 
tfw I realize I just sent 10,000 prompts to 4.1 instead of 4.1-mini and it's gonna cost me \\$20 instead of \\$4 and take a little while. oops!



---

14. [Grok’s ‘white genocide’ meltdown nods to the real dangers of the AI arms race 

There is no unbiased 'view from nowhere'; AI inevitably confirms the biases of its creators, trainers, labelers; in the hands of intemperate, greedy people, it will heavy-handedly favor their agenda and interests.](https://www.cnn.com/2025/05/20/business/grok-genocide-ai-nightcap)  - CNN![image](tmp/Image14.jpg)



---

15. [Driverless vehicles that respond more like humans will cause less harm during road crashes, according to research](https://www.ft.com/content/4eac53c6-85f6-4fe2-8d9c-2e4503473e69)  - Financial Times![image](tmp/Image15.jpg)



---

16. ['Godfather of AI' Geoffrey Hinton says he trusts his chatbot more than he should](https://www.businessinsider.com/geoffrey-hinton-godfather-ai-gpt4-openai-2025)  - Business Insider![image](tmp/Image16.jpg)



---

17. ["the moment I realized AI could code better than me"](https://www.reddit.com/r/ChatGPT/comments/1kq8t4t/the_moment_i_realized_ai_could_code_better_than_me/)  - Reddit

---

18. [Windows is getting support for the MCP, the ‘USB-C of AI apps’](https://www.theverge.com/news/669298/microsoft-windows-ai-foundry-mcp-support)  - The Verge![image](tmp/Image18.jpg)



---

19. 
like all that AI news to catch up on



---

20. [LLMs can beat humans in debates - if they get personal info.

If your candidate's chatbot knows your personal info, maybe it can persuade you better.](https://www.theregister.com/2025/05/19/when_llms_get_personal_info/)  - The Register

---

21. [Time saved by AI offset by new work created, study suggests](https://arstechnica.com/ai/2025/05/time-saved-by-ai-offset-by-new-work-created-study-suggests/)  - Ars Technica![image](tmp/Image21.jpg)



---

22. [SAG-AFTRA filed an unfair labor practice complaint over Darth Vader's voice in Fortnite](https://arstechnica.com/ai/2025/05/fortnites-ai-darth-vader-spawns-unfair-labor-practice-charge-from-voice-union/)  - Ars Technica![image](tmp/Image22.jpg)



---

23. [Trump signs the Take It Down Act criminalizing AI deepfakes today. Hopefully it won't have unintended consequences like putting people in jail for Trump caricatures, sites not allowing image/video posting.](https://www.engadget.com/big-tech/trump-will-sign-the-take-it-down-act-criminalizing-ai-deepfakes-today-184358916.html)  - Engadget![image](tmp/Image23.jpg)



---

24. [Microsoft did some stuff to support registering your agents, monitoring them, and red-teaming them](https://www.zdnet.com/article/microsoft-makes-building-trustworthy-ai-agents-easier-and-more-secure/)  - ZDNET![image](tmp/Image24.jpg)



---

25. 
today I was comparing Substack  v. Ghost v. Beehiiv
tomorrow I might compare NYC mayoral candidates on my hot-button issues

instead of talking to o3, I could have an agent in my browser, that queries the chatbots on the product/campaign pages and synthesizes and gives me the result

brave new world



---

26. [How Google decided to give websites no opt-out/in for AI search](https://www.bloomberg.com/news/articles/2025-05-19/google-gave-sites-little-choice-in-using-data-for-ai-search?srnd=phx-ai)  - Bloomberg

---

27. [Best free AI courses - www.fastcompany.com/91332955/cou...
See also
mashable.com/article/free...
tech.yahoo.com/ai/articles/...
or anything from deeplearning.ai
www.deeplearning.ai/courses/](https://www.fastcompany.com/91332955/coursera-best-free-ai-courses)  - Fast Company

---

28. [A review of brain-computer interfaces, with number of studies and patients set to rise](https://www.wsj.com/tech/brain-implant-musk-als-tbi-neuralink-f733998f)  - The Wall Street Journal

---

29. [New Orleans called out for sketchiest use of facial recognition yet in the US](https://arstechnica.com/tech-policy/2025/05/cops-pause-use-of-flawed-ai-cameras-secretly-monitoring-streets-for-suspects/)  - Ars Technica![image](tmp/Image29.jpg)



---

30. [NLWeb lets you add a chatbot to your web page, and agents would know how to talk to it.](https://techcrunch.com/2025/05/19/nlweb-is-microsofts-project-to-bring-more-chatbots-to-webpages/)  - TechCrunch![image](tmp/Image30.jpg)



---

31. [GitHub allows you to assign issues to a coding agent that will go away and think and submit a PR](https://www.theverge.com/news/669339/github-ai-coding-agent-fix-bugs)  - The Verge![image](tmp/Image31.jpg)



---

32. [An interview with Microsoft CTO Kevin Scott on the birth of the agentic web](https://www.theverge.com/decoder-podcast-with-nilay-patel/669409/microsoft-cto-kevin-scott-interview-ai-natural-language-search-openai)  - The Verge![image](tmp/Image32.jpg)



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [18]:
import markdown
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)


ngl, it generates an excellent fully automated AI news summary tho



---



![image](tmp/Image1.jpg)

[Microsoft launched a scientific AI that screened 100s of thousands of candidates and discovered a potential new coolant in 200 hours instead of years](https://venturebeat.com/ai/microsoft-just-launched-an-ai-that-discovered-a-new-chemical-in-200-hours-instead-of-years/)  - VentureBeat

---



![image](tmp/Image2.jpg)

[Comparing ChatGPT's grammatical judgment with linguists and lay people finds it's pretty good. Not that clear which GPT model. ChatGPT is not great at assigning calibrated scores, comparing 2 sentence is better. (I believe they found that in the study)](https://www.nature.com/articles/s41599-025-04907-8)  - Nature

---



![image](tmp/Image3.jpg)

[techcrunch.com/2025/05/19/k...](https://techcrunch.com/2025/05/19/klarnas-revenue-per-employee-soars-to-nearly-1m-thanks-to-ai-efficiency-push/)  - TechCrunch

---



![image](tmp/Image4.jpg)

[Klarna CEO says AI push may have gone too far, wants more humans now. Pushed revenue per worker way up, but credit quality is down. Who could have predicted someone who buys a DoorDash burrito on a payment plan could be a bad credit risk?](https://futurism.com/klarna-openai-humans-ai-back)  - Futurism

---



![image](tmp/Image5.jpg)

[Stratechery's Ben Thompson Interviews Nvidia CEO Jensen Huang About Chip Controls, AI Factories, and Enterprise Pragmatism](https://stratechery.com/2025/an-interview-with-nvidia-ceo-jensen-huang-about-chip-controls-ai-factories-and-enterprise-pragmatism/)  - Stratechery by Ben Thompson

---



![image](tmp/Image6.jpg)

[Google releases its NotebookLM mobile app](https://www.theverge.com/news/669828/google-releases-its-notebooklm-mobile-app)  - The Verge

---



![image](tmp/Image7.jpg)

[Hyper-contextual, (personalized with facial recognition?) GenAI creative is just about to change the face of out-of-home advertising](https://www.thedrum.com/opinion/2025/05/19/genai-creative-just-about-change-the-face-out-home-advertising)  - The Drum

---



![image](tmp/Image8.jpg)

[Microsoft's big event was all about the 'explosion' of AI agents](https://www.businessinsider.com/microsoft-build-keynote-2025-ai-agent-use-doubled-2025-5)  - Business Insider

---



![image](tmp/Image9.jpg)

[At Build 2025, Microsoft announced major updates to its Copilot Studio platform, including multi-agent computer use that enables different AI agents to collaborate on complex business tasks, integration with WhatsApp, Python-powered analytics, and new developer/admin tools.](https://venturebeat.com/ai/microsoft-just-taught-its-ai-agents-to-talk-to-each-other-and-it-could-transform-how-we-work/)  - VentureBeat

---



![image](tmp/Image10.jpg)

[Baiont’s Feng Ji: Quant managers who don’t adopt AI will be eliminated by the market](https://www.ft.com/content/8eada27c-fa22-4406-9e2b-5034b92642fb)  - Financial Times

---



![image](tmp/Image11.jpg)

[Fair analogy, you have an autopilot and also 2 highly trained pilots monitoring it and turning it on and off at appropriate times. But also, the amount of regulatory oversight over aircraft manufacturing and operation is extremely intense, and every rule was written in blood.](https://www.forbes.com/councils/forbestechcouncil/2025/05/19/if-you-can-trust-an-airplanes-autopilot-you-can-trust-ai/)  - Forbes

---






---


tfw I realize I just sent 10,000 prompts to 4.1 instead of 4.1-mini and it's gonna cost me \\$20 instead of \\$4 and take a little while. oops!



---



![image](tmp/Image14.jpg)

[Grok’s ‘white genocide’ meltdown nods to the real dangers of the AI arms race 

There is no unbiased 'view from nowhere'; AI inevitably confirms the biases of its creators, trainers, labelers; in the hands of intemperate, greedy people, it will heavy-handedly favor their agenda and interests.](https://www.cnn.com/2025/05/20/business/grok-genocide-ai-nightcap)  - CNN

---



![image](tmp/Image15.jpg)

[Driverless vehicles that respond more like humans will cause less harm during road crashes, according to research](https://www.ft.com/content/4eac53c6-85f6-4fe2-8d9c-2e4503473e69)  - Financial Times

---



![image](tmp/Image16.jpg)

['Godfather of AI' Geoffrey Hinton says he trusts his chatbot more than he should](https://www.businessinsider.com/geoffrey-hinton-godfather-ai-gpt4-openai-2025)  - Business Insider

---

["the moment I realized AI could code better than me"](https://www.reddit.com/r/ChatGPT/comments/1kq8t4t/the_moment_i_realized_ai_could_code_better_than_me/)  - Reddit

---



![image](tmp/Image18.jpg)

[Windows is getting support for the MCP, the ‘USB-C of AI apps’](https://www.theverge.com/news/669298/microsoft-windows-ai-foundry-mcp-support)  - The Verge

---


like all that AI news to catch up on



---

[LLMs can beat humans in debates - if they get personal info.

If your candidate's chatbot knows your personal info, maybe it can persuade you better.](https://www.theregister.com/2025/05/19/when_llms_get_personal_info/)  - The Register

---



![image](tmp/Image21.jpg)

[Time saved by AI offset by new work created, study suggests](https://arstechnica.com/ai/2025/05/time-saved-by-ai-offset-by-new-work-created-study-suggests/)  - Ars Technica

---



![image](tmp/Image22.jpg)

[SAG-AFTRA filed an unfair labor practice complaint over Darth Vader's voice in Fortnite](https://arstechnica.com/ai/2025/05/fortnites-ai-darth-vader-spawns-unfair-labor-practice-charge-from-voice-union/)  - Ars Technica

---



![image](tmp/Image23.jpg)

[Trump signs the Take It Down Act criminalizing AI deepfakes today. Hopefully it won't have unintended consequences like putting people in jail for Trump caricatures, sites not allowing image/video posting.](https://www.engadget.com/big-tech/trump-will-sign-the-take-it-down-act-criminalizing-ai-deepfakes-today-184358916.html)  - Engadget

---



![image](tmp/Image24.jpg)

[Microsoft did some stuff to support registering your agents, monitoring them, and red-teaming them](https://www.zdnet.com/article/microsoft-makes-building-trustworthy-ai-agents-easier-and-more-secure/)  - ZDNET

---


today I was comparing Substack  v. Ghost v. Beehiiv
tomorrow I might compare NYC mayoral candidates on my hot-button issues

instead of talking to o3, I could have an agent in my browser, that queries the chatbots on the product/campaign pages and synthesizes and gives me the result

brave new world



---

[How Google decided to give websites no opt-out/in for AI search](https://www.bloomberg.com/news/articles/2025-05-19/google-gave-sites-little-choice-in-using-data-for-ai-search?srnd=phx-ai)  - Bloomberg

---

[Best free AI courses - www.fastcompany.com/91332955/cou...
See also
mashable.com/article/free...
tech.yahoo.com/ai/articles/...
or anything from deeplearning.ai
www.deeplearning.ai/courses/](https://www.fastcompany.com/91332955/coursera-best-free-ai-courses)  - Fast Company

---

[A review of brain-computer interfaces, with number of studies and patients set to rise](https://www.wsj.com/tech/brain-implant-musk-als-tbi-neuralink-f733998f)  - The Wall Street Journal

---



![image](tmp/Image29.jpg)

[New Orleans called out for sketchiest use of facial recognition yet in the US](https://arstechnica.com/tech-policy/2025/05/cops-pause-use-of-flawed-ai-cameras-secretly-monitoring-streets-for-suspects/)  - Ars Technica

---



![image](tmp/Image30.jpg)

[NLWeb lets you add a chatbot to your web page, and agents would know how to talk to it.](https://techcrunch.com/2025/05/19/nlweb-is-microsofts-project-to-bring-more-chatbots-to-webpages/)  - TechCrunch

---



![image](tmp/Image31.jpg)

[GitHub allows you to assign issues to a coding agent that will go away and think and submit a PR](https://www.theverge.com/news/669339/github-ai-coding-agent-fix-bugs)  - The Verge

---



![image](tmp/Image32.jpg)

[An interview with Microsoft CTO Kevin Scott on the birth of the agentic web](https://www.theverge.com/decoder-podcast-with-nilay-patel/669409/microsoft-cto-kevin-scott-interview-ai-natural-language-search-openai)  - The Verge

---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

9272

In [19]:
from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(RootModel[list[int]]):
    """List of story indexes"""
    pass


In [20]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import RootModel
import os

from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")


system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most imporant story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

user_prompt = """
{input_text}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

model = ChatOpenAI(
        model="gpt-4.1",  
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
# Create the chain
chain = prompt_template | model.with_structured_output(ItemList)
response = chain.invoke({"input_text": output_df.to_json(orient="records")})


In [21]:
len(response.items)


33

In [22]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    



![image](tmp/Image23.jpg)

[Trump signs the Take It Down Act criminalizing AI deepfakes today. Hopefully it won't have unintended consequences like putting people in jail for Trump caricatures, sites not allowing image/video posting.](https://www.engadget.com/big-tech/trump-will-sign-the-take-it-down-act-criminalizing-ai-deepfakes-today-184358916.html)  - Engadget

---

[How Google decided to give websites no opt-out/in for AI search](https://www.bloomberg.com/news/articles/2025-05-19/google-gave-sites-little-choice-in-using-data-for-ai-search?srnd=phx-ai)  - Bloomberg

---



![image](tmp/Image6.jpg)

[Google releases its NotebookLM mobile app](https://www.theverge.com/news/669828/google-releases-its-notebooklm-mobile-app)  - The Verge

---



![image](tmp/Image18.jpg)

[Windows is getting support for the MCP, the ‘USB-C of AI apps’](https://www.theverge.com/news/669298/microsoft-windows-ai-foundry-mcp-support)  - The Verge

---



![image](tmp/Image30.jpg)

[NLWeb lets you add a chatbot to your web page, and agents would know how to talk to it.](https://techcrunch.com/2025/05/19/nlweb-is-microsofts-project-to-bring-more-chatbots-to-webpages/)  - TechCrunch

---



![image](tmp/Image8.jpg)

[Microsoft's big event was all about the 'explosion' of AI agents](https://www.businessinsider.com/microsoft-build-keynote-2025-ai-agent-use-doubled-2025-5)  - Business Insider

---



![image](tmp/Image9.jpg)

[At Build 2025, Microsoft announced major updates to its Copilot Studio platform, including multi-agent computer use that enables different AI agents to collaborate on complex business tasks, integration with WhatsApp, Python-powered analytics, and new developer/admin tools.](https://venturebeat.com/ai/microsoft-just-taught-its-ai-agents-to-talk-to-each-other-and-it-could-transform-how-we-work/)  - VentureBeat

---



![image](tmp/Image24.jpg)

[Microsoft did some stuff to support registering your agents, monitoring them, and red-teaming them](https://www.zdnet.com/article/microsoft-makes-building-trustworthy-ai-agents-easier-and-more-secure/)  - ZDNET

---



![image](tmp/Image32.jpg)

[An interview with Microsoft CTO Kevin Scott on the birth of the agentic web](https://www.theverge.com/decoder-podcast-with-nilay-patel/669409/microsoft-cto-kevin-scott-interview-ai-natural-language-search-openai)  - The Verge

---



![image](tmp/Image1.jpg)

[Microsoft launched a scientific AI that screened 100s of thousands of candidates and discovered a potential new coolant in 200 hours instead of years](https://venturebeat.com/ai/microsoft-just-launched-an-ai-that-discovered-a-new-chemical-in-200-hours-instead-of-years/)  - VentureBeat

---



![image](tmp/Image3.jpg)

[techcrunch.com/2025/05/19/k...](https://techcrunch.com/2025/05/19/klarnas-revenue-per-employee-soars-to-nearly-1m-thanks-to-ai-efficiency-push/)  - TechCrunch

---



![image](tmp/Image4.jpg)

[Klarna CEO says AI push may have gone too far, wants more humans now. Pushed revenue per worker way up, but credit quality is down. Who could have predicted someone who buys a DoorDash burrito on a payment plan could be a bad credit risk?](https://futurism.com/klarna-openai-humans-ai-back)  - Futurism

---



![image](tmp/Image10.jpg)

[Baiont’s Feng Ji: Quant managers who don’t adopt AI will be eliminated by the market](https://www.ft.com/content/8eada27c-fa22-4406-9e2b-5034b92642fb)  - Financial Times

---



![image](tmp/Image21.jpg)

[Time saved by AI offset by new work created, study suggests](https://arstechnica.com/ai/2025/05/time-saved-by-ai-offset-by-new-work-created-study-suggests/)  - Ars Technica

---



![image](tmp/Image22.jpg)

[SAG-AFTRA filed an unfair labor practice complaint over Darth Vader's voice in Fortnite](https://arstechnica.com/ai/2025/05/fortnites-ai-darth-vader-spawns-unfair-labor-practice-charge-from-voice-union/)  - Ars Technica

---



![image](tmp/Image29.jpg)

[New Orleans called out for sketchiest use of facial recognition yet in the US](https://arstechnica.com/tech-policy/2025/05/cops-pause-use-of-flawed-ai-cameras-secretly-monitoring-streets-for-suspects/)  - Ars Technica

---



![image](tmp/Image7.jpg)

[Hyper-contextual, (personalized with facial recognition?) GenAI creative is just about to change the face of out-of-home advertising](https://www.thedrum.com/opinion/2025/05/19/genai-creative-just-about-change-the-face-out-home-advertising)  - The Drum

---

[LLMs can beat humans in debates - if they get personal info.

If your candidate's chatbot knows your personal info, maybe it can persuade you better.](https://www.theregister.com/2025/05/19/when_llms_get_personal_info/)  - The Register

---



![image](tmp/Image2.jpg)

[Comparing ChatGPT's grammatical judgment with linguists and lay people finds it's pretty good. Not that clear which GPT model. ChatGPT is not great at assigning calibrated scores, comparing 2 sentence is better. (I believe they found that in the study)](https://www.nature.com/articles/s41599-025-04907-8)  - Nature

---



![image](tmp/Image16.jpg)

['Godfather of AI' Geoffrey Hinton says he trusts his chatbot more than he should](https://www.businessinsider.com/geoffrey-hinton-godfather-ai-gpt4-openai-2025)  - Business Insider

---

["the moment I realized AI could code better than me"](https://www.reddit.com/r/ChatGPT/comments/1kq8t4t/the_moment_i_realized_ai_could_code_better_than_me/)  - Reddit

---



![image](tmp/Image31.jpg)

[GitHub allows you to assign issues to a coding agent that will go away and think and submit a PR](https://www.theverge.com/news/669339/github-ai-coding-agent-fix-bugs)  - The Verge

---



![image](tmp/Image11.jpg)

[Fair analogy, you have an autopilot and also 2 highly trained pilots monitoring it and turning it on and off at appropriate times. But also, the amount of regulatory oversight over aircraft manufacturing and operation is extremely intense, and every rule was written in blood.](https://www.forbes.com/councils/forbestechcouncil/2025/05/19/if-you-can-trust-an-airplanes-autopilot-you-can-trust-ai/)  - Forbes

---

[A review of brain-computer interfaces, with number of studies and patients set to rise](https://www.wsj.com/tech/brain-implant-musk-als-tbi-neuralink-f733998f)  - The Wall Street Journal

---

[Best free AI courses - www.fastcompany.com/91332955/cou...
See also
mashable.com/article/free...
tech.yahoo.com/ai/articles/...
or anything from deeplearning.ai
www.deeplearning.ai/courses/](https://www.fastcompany.com/91332955/coursera-best-free-ai-courses)  - Fast Company

---



![image](tmp/Image14.jpg)

[Grok’s ‘white genocide’ meltdown nods to the real dangers of the AI arms race 

There is no unbiased 'view from nowhere'; AI inevitably confirms the biases of its creators, trainers, labelers; in the hands of intemperate, greedy people, it will heavy-handedly favor their agenda and interests.](https://www.cnn.com/2025/05/20/business/grok-genocide-ai-nightcap)  - CNN

---



![image](tmp/Image15.jpg)

[Driverless vehicles that respond more like humans will cause less harm during road crashes, according to research](https://www.ft.com/content/4eac53c6-85f6-4fe2-8d9c-2e4503473e69)  - Financial Times

---



![image](tmp/Image5.jpg)

[Stratechery's Ben Thompson Interviews Nvidia CEO Jensen Huang About Chip Controls, AI Factories, and Enterprise Pragmatism](https://stratechery.com/2025/an-interview-with-nvidia-ceo-jensen-huang-about-chip-controls-ai-factories-and-enterprise-pragmatism/)  - Stratechery by Ben Thompson

---


ngl, it generates an excellent fully automated AI news summary tho



---






---


today I was comparing Substack  v. Ghost v. Beehiiv
tomorrow I might compare NYC mayoral candidates on my hot-button issues

instead of talking to o3, I could have an agent in my browser, that queries the chatbots on the product/campaign pages and synthesizes and gives me the result

brave new world



---


tfw I realize I just sent 10,000 prompts to 4.1 instead of 4.1-mini and it's gonna cost me \\$20 instead of \\$4 and take a little while. oops!



---


like all that AI news to catch up on



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

9272

In [ ]:
pyperclip.copy( output_str )


In [ ]:
date_str = data.feed[0].post.record.created_at
dt = datetime.fromisoformat(date_str.rstrip('Z'))
dt

In [ ]:

# Get the current datetime
now = datetime.now()

# Create a timedelta of 30 hours
delta = timedelta(hours=30)

# Subtract 30 hours from the current datetime
thirty_hours_ago = now - delta

print(thirty_hours_ago)


In [ ]:
from typing import TypedDict, Annotated

class Point2D(TypedDict, total=False):
    x: int
    y: int
    label: str

a: Point2D = {'x': 1, 'y': 2, }  # 


In [ ]:
a

In [ ]:
import numpy as np

def decay_exponential(t, N0):
    k = np.log(2)  
    return N0 * np.exp(-k * t)

2*decay_exponential(4,1)-1

In [ ]:
```mermaid
flowchart LR

A[Hard] -->|Text| B(Round)
B --> C{Decision}
C -->|One| D[Result 1]
C -->|Two| E[Result 2]
